**Read the Equipment Sensor Streaming Data from One Lake**

In [3]:
delta_table_path = "abfss://FabricDemo@onelake.dfs.fabric.microsoft.com/Equipment_Sensor_Lakehouse.Lakehouse/Tables/Equipment-Sensor-Streaming"
equipmentSensorStreamingRaw = spark.read.format("delta").load(delta_table_path)
equipmentSensorStreamingRaw.createOrReplaceTempView("equipmentSensorStreamingRaw")
equipmentSensorStreamingRaw.show()


StatementMeta(, df7ef874-c4dd-4c9e-ba56-6e1f0ee2cef1, 5, Finished, Available, Finished)

+--------------------+--------+--------------------+-----------+--------------+--------------+-------+---+-------------+
|            sensorid|deviceid|    readingtimestamp|temperature|vibrationlevel|operatinghours| status| op|        ts_ms|
+--------------------+--------+--------------------+-----------+--------------+--------------+-------+---+-------------+
|d7d0db14-41f9-4a4...| sen_008|2024-11-09 18:16:...|      54.82|          3.34|           124| Normal|  c|1731156392899|
|9b448104-87b7-447...| sen_007|2024-11-09 18:16:...|      73.82|          0.78|           272|Warning|  c|1731156392899|
|71cd95a1-7f7a-419...| sen_009|2024-11-09 18:16:...|      50.78|           1.5|           226| Normal|  c|1731156392900|
|fbc919be-69a1-482...| sen_010|2024-11-09 18:16:...|      76.42|          6.89|            13|Warning|  c|1731156392900|
|3d86cb28-822e-451...| sen_006|2024-11-09 18:16:...|      64.33|          5.45|           155|Warning|  c|1731156392899|
|87f102b9-13aa-4cf...| sen_011|2

**Aggregate the Data to the Day Grain**

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window


windowSpec = Window.partitionBy("DEVICEID", F.col("readingtimestamp").cast("date")) \
                   .orderBy(F.col("readingtimestamp").desc())
equipmentSensorStreamingAgg = (\
    spark.table("equipmentSensorStreamingRaw")\
    .select(
        "DEVICEID",
        F.col("readingtimestamp").cast("date").alias("readingtimestamp"),
        F.last("temperature", True).over(windowSpec).alias("temperature"),
        F.last("vibrationlevel", True).over(windowSpec).alias("vibrationlevel"),
        F.avg("operatinghours").over(windowSpec).alias("operatinghours"),
        F.last("status", True).over(windowSpec).alias("status")
    )\
    .groupBy("DEVICEID", "readingtimestamp")\
    .agg(
        F.first("temperature").alias("temperature"),
        F.first("vibrationlevel").alias("vibrationlevel"),
        F.avg("operatinghours").alias("operatinghours"),
        F.first("status").alias("status")
    )\
)
equipmentSensorStreamingAgg.show()


StatementMeta(, df7ef874-c4dd-4c9e-ba56-6e1f0ee2cef1, 15, Finished, Available, Finished)

+--------+----------------+-----------+--------------+--------------+-------+
|DEVICEID|readingtimestamp|temperature|vibrationlevel|operatinghours| status|
+--------+----------------+-----------+--------------+--------------+-------+
| sen_006|      2024-11-09|      65.24|          1.21|          66.5|Warning|
| sen_007|      2024-11-09|      78.37|           1.1|         234.5|Warning|
| sen_008|      2024-11-09|      73.49|          1.91|        396.25|Warning|
| sen_009|      2024-11-09|      68.79|           6.2|         338.5|Warning|
| sen_010|      2024-11-09|       52.8|           6.7|         251.5|Warning|
| sen_010|      2024-11-10|      79.69|          0.96|          59.0|Warning|
| sen_011|      2024-11-10|       62.0|          1.27|           0.0|Warning|
| sen_012|      2024-11-10|      54.96|          5.42|          51.0|Warning|
| sen_013|      2024-11-10|      53.26|          5.69|         353.0|Warning|
| sen_014|      2024-11-10|      20.78|          1.36|         4

**Write the aggregated data to lakehouse**

In [6]:
delta_target_path = "abfss://FabricDemo@onelake.dfs.fabric.microsoft.com/Equipment_Sensor_Lakehouse.Lakehouse/Files"
equipmentSensorStreamingAgg.write.format("delta").mode("append").saveAsTable("EQUIPMENT_SENSOR_SNAPSHOT_DAILY", path=delta_target_path)

StatementMeta(, 25f48f30-4ab7-4822-b05a-f89644ae4f50, 8, Finished, Available, Finished)